## exemple de Code :

<pre style='color:#000020;background:#f6f8ff;'>
<span style='color:#200080; font-weight:bold; '>begin</span>

    <span style='color:#200080; font-weight:bold; '>let</span> _a <span style='color:#308080; '>=</span> <span style='color:#008c00; '>1;</span>
    <span style='color:#200080; font-weight:bold; '>let</span> _b <span style='color:#308080; '>=</span> <span style='color:#008c00; '>1;</span>

    <span style='color:#200080; font-weight:bold; '>let</span> _i <span style='color:#308080; '>=</span> <span style='color:#008c00; '>1;</span>
    <span style='color:#200080; font-weight:bold; '>let</span> _n<span style='color:#308080; '>;</span>

    _n <span style='color:#308080; '>></span><span style='color:#308080; '>>;</span>    <span style='color:#595979; '>// lire(n)</span>

    <span style='color:#200080; font-weight:bold; '>while</span> <span style='color:#308080; '>(</span>_i <span style='color:#308080; '>&lt;</span> _n<span style='color:#308080; '>)</span>:

        _x <span style='color:#308080; '>=</span> _a <span style='color:#308080; '>+</span> _b<span style='color:#308080; '>;</span>
        _a <span style='color:#308080; '>=</span> _b<span style='color:#308080; '>;</span>
        _b <span style='color:#308080; '>=</span> _x<span style='color:#308080; '>;</span>

    _i<span style='color:#308080; '> =</span> _i<span style='color:#308080; '> +</span><span style='color:#008c00; '> 1</span><span style='color:#308080; '>;</span>
    <span style='color:#200080; font-weight:bold; '>ewhile</span>

    _x <span style='color:#308080; '>&lt;</span><span style='color:#308080; '>&lt;;</span>

<span style='color:#200080; font-weight:bold; '>end</span>
</pre>


# Analyse Lexicale :

tout d'abord on defini le langage comme suit :

## les mots clé :

- begin
- end


- let


- if
- eif


- else
- eelse


- while
- ewhile


### et leurs automate : 

![alt text](keywords_automata.png)

## les noms de variables :

 > Ce langage accepte les noms de variables qui commancent par ** `_` **

### et leurs automate :

![alt text](id_names.png)

## les operateurs :

In [15]:
oprt = ['-','*','+','/','%','=','~','>','<']
# ~ <=> ==

# on ajout apres:
# >> read
# << write
# >=   <=


## les séparateurs :

In [16]:
sep = ['(',')',';',':']

### codification( a utiliser dans l'analyse syntaxique ) :

In [17]:
ADD   = 0    # +
SUB   = 1    # -
MUL   = 2    # *
DIV   = 3    # /
MOD   = 4    # %
AFF   = 5    # =
EQ    = 6    # ~

LE    = 7   # <
LEQ   = 8   # <=

GT    = 9   # >
GTQ   = 10   # >=

READ  = 11   # >>
PRINT = 12   # <<

SEP_OUV = 13 # (
SEP_FER = 14 # )
SEMI    = 15 # ;
DPNT    = 16

ID      = 17 # idf
CONST   = 18


KEY_BEG   =19 # begin
KEY_END   =20 # end

KEY_IF    =21 # if
KEY_EIF   =22
KEY_ELSE  =23 # else
KEY_EELSE = 24

KEY_WH    =25 # while
KEY_EWH =26 # ewhile

KEY_LET   =27  # let


## les constantes :

les constantes signifie tout nombre ( type double dans le langage C++)
> les constantes ont la forme {0:9}.{.}.{0:9}

In [18]:
code = """begin
& declaration 
let _a = 1;
let _b = 1;
    
let _i = 0;
let _n;

& lecture :    
_n >>;
    
while (_i <= _n)
    
_x = _a + _b;
_a = _b;
_b = _x;
    
_i = _i + 1;
ewhile;

& affichage :  
_x <<;

end


"""

# Partie : Implémentation

In [1]:
import numpy as np
import pandas as pd
import string
import sys

In [20]:
keywords_finit_stats = [5]
keywords_automata = pd.read_csv('keywords.csv')

### les fonctions de base : 

In [21]:
def is_keyword(string):
    string += '#'
    i = 0
    current_state = 0
    current_term = string[0]
    
    while current_term != '#' and current_state != -1 :
        current_state = keywords_automata[string[i]][current_state]
        i+=1
        current_term = string[i]        
        
    if current_term == "#" and current_state in keywords_finit_stats:
        return True   # accepted
    else :
        return False  # not accepted

In [22]:
def is_idf(string):
    i=0
    if(string[i] == '_'):
        return string[1:].isalnum()
    else:
        return False

In [23]:
def is_const(string):
    i=0
    if(string[i] == '.' or string[len(string) - 1] == '.'):
        return False
    else:
        num = string.replace('.', '')
        return num.isdigit() and ( string.count('.') == 1 or string.count('.') == 0 )

In [24]:
def error(msg='',sym=' ',ligne=0):
    print("Error :" + msg +" "+ sym + " at ligne " + str(ligne))
    sys.exit(1)

In [25]:
def tok(token_type,token):
    """
    input : tocken_type , token
    output : dict of token
    """
    return {"token_type" : token_type , "value" : token}

### les tests :

In [26]:
#keywords_automata

In [27]:
is_keyword('let')

True

In [28]:
is_idf('_LArynx1')

True

In [29]:
is_const('1.A2')

False

## Analyseur lexical : 

In [30]:
def lexical(code):
    i = 0
    num_ligne = 1;
    tokens_table = []
    while i < len(code):
        # calcule de lign :::::::::::::::::::::::::::::::::::::::::::::::::::::
        if(code[i] == '\n') : num_ligne += 1
        # print(num_ligne)


        # comments  :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
        if code[i]=='&':
            while code[i] != '\n':
                i+=1


        # skip spaces  ::::::::::::::::::::::::::::::::::::::::::::::::::::::::
        elif code[i].isspace():
            pass


        # operateur  ::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
        elif code[i] in oprt:
            if  code[i] == '+' : 
                tokens_table.append(tok(ADD,None))
                #print(code[i])
            elif code[i]== '-' : 
                tokens_table.append(tok(SUB,None))
                #print(code[i])
            elif code[i]== '*' : 
                tokens_table.append(tok(MUL,None))
                #print(code[i])
            elif code[i]== '/' : 
                tokens_table.append(tok(DIV,None))
                #print(code[i])
            elif code[i]== '%' : 
                tokens_table.append(tok(MOD,None))
                #print(code[i])
            elif code[i]== '=' : 
                tokens_table.append(tok(AFF,None))
                #print(code[i])
            elif code[i]== '~' : 
                tokens_table.append(tok(EQ,None))
                #print(code[i])

            elif code[i]== '<' :
                if code[i+1] == '<':
                    tokens_table.append(tok(PRINT,None))
                    #print(code[i]+code[i+1])
                    i+=1
                elif code[i+1] == '=':
                    tokens_table.append(tok(LEQ,None)) # <=
                    #print(code[i]+code[i+1])
                    i+=1
                else :
                    tokens_table.append(tok(LE,None))  # <
                    #print(code[i])


            elif code[i]== '>' :
                if code[i+1] == '>':
                    tokens_table.append(tok(READ,None))
                    #print(code[i]+code[i+1])
                    i+=1

                elif code[i+1] == '=':
                    tokens_table.append(tok(GTQ,None)) # <=
                    #print(code[i]+code[i+1])
                    i+=1

                else :
                    tokens_table.append(tok(GT,None)) # <
                    #print(code[i])


        # separateur :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::       
        elif code[i] in sep :
            if code[i] == '(': 
                tokens_table.append(tok(SEP_OUV,None))
            elif code[i] == ')':
                tokens_table.append(tok(SEP_OUV,None))
            elif code[i] == ';':
                tokens_table.append(tok(SEMI,None))
            elif code[i] == ':':
                tokens_table.append(tok(DPNT,None))
            #print(code[i]) # afficher supp      


        # idf ::::::::::::::::::::::::::::::::::::::::::::::::::::::;:::::::::::::
        elif code[i] == '_':
            idf=''
            while (not code[i].isspace())and(code[i] not in oprt)and(code[i] not in sep):
                idf += code[i]
                i+=1
            if is_idf(idf):
                tokens_table.append(tok(ID,idf))
                # print(idf)
            else :
                error("INVALIDE IDF",idf,num_ligne) # : error ligne number !
            i-=1   


        # keywords ::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
        elif code[i].isalpha():
            word = ''
            while not code[i].isspace() and  code[i] not in sep:
                word += code[i]
                i+=1
            #print(word)
            if is_keyword(word):
                if   word == "begin" :tokens_table.append(tok(KEY_BEG,word))
                elif word == "end" :tokens_table.append(tok(KEY_END,word))
                elif word == "if" :tokens_table.append(tok(KEY_IF,word))
                elif word == "eif" :tokens_table.append(tok(KEY_EIF,word))
                elif word == "else" :tokens_table.append(tok(KEY_ELSE,word))
                elif word == "eelse" :tokens_table.append(tok(KEY_EELSE,word))
                elif word == "while" :tokens_table.append(tok(KEY_WH,word))
                elif word == "ewhile" :tokens_table.append(tok(KEY_EWH,word))
                elif word == "let" :tokens_table.append(tok(KEY_LET,word))     
            else :
                error("INVALIDE KEYWORD",word,num_ligne) # : error ligne number !

            # const (numero) ::::::::::::::::::::::::::::::::::::::::
        elif code[i].isnumeric():
            num = ''
            while not code[i].isspace() and code[i] not in sep: # add not in oprt
                num += code[i]
                i+=1
            #print(num)
            i-=1 # <----
            if is_const(num):
                tokens_table.append(tok(CONST,num))
            else :
                error("INVALIDE CONST",num,num_ligne)


        else:
            # num_ligne = 120
            error("INVALIDE SYMBOLE",code[i],num_ligne) 
        i+=1
    return tokens_table

In [31]:
output = lexical(code)

In [32]:
output # TODO : replace codifivcation with negativer integer !!

[{'token_type': 19, 'value': 'begin'},
 {'token_type': 27, 'value': 'let'},
 {'token_type': 17, 'value': '_a'},
 {'token_type': 5, 'value': None},
 {'token_type': 18, 'value': '1'},
 {'token_type': 15, 'value': None},
 {'token_type': 27, 'value': 'let'},
 {'token_type': 17, 'value': '_b'},
 {'token_type': 5, 'value': None},
 {'token_type': 18, 'value': '1'},
 {'token_type': 15, 'value': None},
 {'token_type': 27, 'value': 'let'},
 {'token_type': 17, 'value': '_i'},
 {'token_type': 5, 'value': None},
 {'token_type': 18, 'value': '0'},
 {'token_type': 15, 'value': None},
 {'token_type': 27, 'value': 'let'},
 {'token_type': 17, 'value': '_n'},
 {'token_type': 15, 'value': None},
 {'token_type': 17, 'value': '_n'},
 {'token_type': 11, 'value': None},
 {'token_type': 15, 'value': None},
 {'token_type': 25, 'value': 'while'},
 {'token_type': 13, 'value': None},
 {'token_type': 17, 'value': '_i'},
 {'token_type': 8, 'value': None},
 {'token_type': 17, 'value': '_n'},
 {'token_type': 13, 'valu

# Analyse Syntaxique

## Grammaire

PROG -> begin BODY end  
BODY -> VARDCLR BODY  
BODY -> INSTRUCTIONS BODY  
BODY -> ''  
  
INSTRUCTIONS -> AFF  
INSTRUCTIONS-> WHILE  
INSTRUCTIONS-> IF  
INSTRUCTIONS-> IN  
INSTRUCTIONS -> OUT  
  
VARDCLR -> let IDF STOP  
STOP -> ;  
STOP -> = idf ;  
STOP -> = num ;  
  
EXPR -> T X  
X -> + T X  
X -> - T X  

X -> ''  
T -> F Z  
Z -> * F Z  
Z -> / F Z  
Z -> ''  
F -> ( EXPR )  
F -> idf  
F -> num  
  
AFF -> IDF = EXPR ;  
WHILE -> while ( COND ) : BODY ewhile  
COND -> OPRD OPR OPRD  
OPR -> ~  
OPR -> <  
OPR -> <=  
OPR -> >  
OPR -> >=  
OPRD -> IDF  
OPRD -> NUM  
  
IF -> if ( COND ) : BODY eif else : BODY eelse  
IF -> if ( COND ) : BODY eif  
  
IN -> >> IDF ;  
OUT -> << OPRD ;  
IDF -> idf  
NUM -> num  

## table des débuts :

| Nonterminal  | FIRST                     |
|:-------------|:------------------------- |
| PROG         | begin                     |
| BODY         | ε,let,while,if,>>,<<,idf |
| INSTRUCTIONS | while,if,>>,<<,idf        |
| VARDCLR      | let                       |
| STOP         | ;,=                       |
| EXPR         | \(,idf,num                |
| X            | \+,\-,ε                  |
| T            | \(,idf,num                |
| Z            | \*,/,ε                   |
| F            | \(,idf,num                |
| AFF          | idf                       |
| WHILE        | while                     |
| COND         | idf,num                   |
| OPR          | ~,<,<=,>,>=               |
| OPRD         | idf,num                   |
| IF           | if                        |
| IN           | >>                        |
| OUT          | <<                        |
| IDF          | idf                       |
| NUM          | num                       |


### Grammar implementation :

In [45]:
def get_reduction_rule(number):
    grammar = {
        0:{"non_terminal" : "PROG", "production" : "begin BODY end"},  
        1:{"non_terminal" : "BODY", "production" : "VARDCLR BODY"},  
        2:{"non_terminal" : "BODY", "production" : "INSTRUCTIONS BODY"},  
        3:{"non_terminal" : "BODY", "production" : ""},  

        4:{"non_terminal" : "INSTRUCTIONS", "production" : "AFF"},  
        5:{"non_terminal" : "INSTRUCTIONS", "production" : "WHILE"},  
        6:{"non_terminal" : "INSTRUCTIONS", "production" : "IF"},  
        7:{"non_terminal" : "INSTRUCTIONS", "production" : "IN"},  
        8:{"non_terminal" : "INSTRUCTIONS", "production" : "OUT"},  

        9:{"non_terminal" : "VARDCLR", "production" : "let IDF STOP"},  
        10:{"non_terminal" : "STOP", "production" : ";"},  
        11:{"non_terminal" : "STOP", "production" : "= idf ;"},  
        12:{"non_terminal" : "STOP", "production" : "= num ;"},  

        13:{"non_terminal" : "EXPR", "production" : "T X"},  
        14:{"non_terminal" : "X", "production" : "+ T X"},  
        15:{"non_terminal" : "X", "production" : "- T X"},  

        16:{"non_terminal" : "X", "production" : ""},  
        17:{"non_terminal" : "T", "production" : "F Z"},  
        18:{"non_terminal" : "Z", "production" : "* F Z"},  
        19:{"non_terminal" : "Z", "production" : "/ F Z"},  
        20:{"non_terminal" : "Z", "production" : ""},  
        21:{"non_terminal" : "F", "production" : "( EXPR )"},  
        22:{"non_terminal" : "F", "production" : "idf"},  
        23:{"non_terminal" : "F", "production" : "num"},  

        24:{"non_terminal" : "AFF", "production" : "IDF = EXPR ;"},  
        25:{"non_terminal" : "WHILE", "production" : "while ( COND ) : BODY ewhile"},  
        26:{"non_terminal" : "COND", "production" : "OPRD OPR OPRD"},  
        27:{"non_terminal" : "OPR", "production" : "~"},  
        28:{"non_terminal" : "OPR", "production" : "<"},  
        29:{"non_terminal" : "OPR", "production" : "<="},  
        30:{"non_terminal" : "OPR", "production" : ">"},  
        31:{"non_terminal" : "OPR", "production" : ">="},  
        32:{"non_terminal" : "OPRD", "production" : "IDF"},  
        33:{"non_terminal" : "OPRD", "production" : "NUM"},  

        34:{"non_terminal" : "IF", "production" : "if ( COND ) : BODY eif else : BODY eelse"},  
        35:{"non_terminal" : "IF", "production" : "if ( COND ) : BODY eif"},  

        36:{"non_terminal" : "IN",  "production" : ">> IDF ;"},  
        37:{"non_terminal" : "OUT", "production" : "<< OPRD ;"},  
        38:{"non_terminal" : "IDF", "production" : "idf"},  
        39:{"non_terminal" : "NUM", "production" : "num"}  

    }
    
    return grammar.get(number,"Pas d'action")
    

In [48]:
get_reduction_rule(1)

{'non_terminal': 'BODY', 'production': 'VARDCLR BODY'}

In [47]:
lr_table = pd.read_csv("lalr/LR_table.csv")

In [8]:
lr_table["$"][17]

'acc'

In [13]:
lr_table["begin"][0]

's1'

### test de l'analyse 

In [19]:
def get_action(action):
    if action == "acc"  :
        return {"action": "accept",
                "number":None}
    
    elif action[0] == "r" :
        return {"action": "Reduction",
                "number":action[1:]}
    
    elif action[0] == "s" :
        return {"action": "Shift",
                "number": action[1:]}
    
    elif action == ' ' :
        return {"action": None,
                "number":None}

In [84]:
# test !:::::::
prog = "begin >> idf end"

In [51]:
prog = "begin let idf = num end"

In [90]:
prog = "begin let idf = num ; let idf = num ; let idf = num ; let idf ; >> idf ; while ( idf <= idf ) : idf = idf + idf ; idf = idf ; idf = idf ; idf = idf + num ; ewhile << idf ; end"

In [91]:
chaine = prog.split(sep=" ")
chaine += "$"

Stack = ['#',0]
#init
ptr = chaine[0]
state = Stack[-1] # state = 0
Do = get_action( lr_table[ptr][0] )

# then do 
while (Do["action"] != None and chaine != ['$']):
    haja = []
    if   Do["action"] == "Shift" : 
        # SHIFT ==============================::
        # D4 => S4 => shift_number = 4
        number = int(Do["number"])
        Stack.append( chaine.pop(0) )
        Stack.append(number)     

        # REDUCTION ==========================::
    elif Do["action"] == "Reduction":
        number = int( Do["number"] )        
        regle = get_reduction_rule(number)
        
        print(regle)
        
        
        right_part = regle["production"].split(sep=" ")

        right_part.reverse()
        left_part = regle["non_terminal"]
        
        #:faire la réducyion:#
        while ( right_part != [] ):
            if right_part == [''] :
                break
            if Stack[-1] != right_part[0] :
                Stack.pop()
            elif Stack[-1] == right_part[0]: 
                Stack.pop()
                haja += right_part[0]
                right_part.pop(0)
            else :
                sys.exit(0)
        # get la relation entre Stack[-1] et le non-terminal 
        rel = lr_table[left_part][Stack[-1]]
        # empiler la partie gauche :
        Stack.append(left_part)
        # empiler la relation
        Stack.append(int(rel)) 
    print(Stack)
    #print("-------")
    #print(chaine)
        # NEXT ===============================::
    Do = get_action(lr_table[chaine[0]][int(Stack[-1])])

if(Do["action"] == "accept" and chaine == ['$']) : 
    print("=============================")
    print("entité syntaxiquement CORRECT")
    print("=============================")
else :
    print("I will handle errors soon ! ")
    print(Do["number"])

['#', 0, 'begin', 1]
['#', 0, 'begin', 1, 'let', 5]
['#', 0, 'begin', 1, 'let', 5, 'idf', 16]
{'non_terminal': 'IDF', 'production': 'idf'}
['#', 0, 'begin', 1, 'let', 5, 'IDF', 20]
['#', 0, 'begin', 1, 'let', 5, 'IDF', 20, '=', 31]
['#', 0, 'begin', 1, 'let', 5, 'IDF', 20, '=', 31, 'num', 44]
['#', 0, 'begin', 1, 'let', 5, 'IDF', 20, '=', 31, 'num', 44, ';', 62]
{'non_terminal': 'STOP', 'production': '= num ;'}
['#', 0, 'begin', 1, 'let', 5, 'IDF', 20, 'STOP', 29]
{'non_terminal': 'VARDCLR', 'production': 'let IDF STOP'}
['#', 0, 'begin', 1, 'VARDCLR', 3]
['#', 0, 'begin', 1, 'VARDCLR', 3, 'let', 5]
['#', 0, 'begin', 1, 'VARDCLR', 3, 'let', 5, 'idf', 16]
{'non_terminal': 'IDF', 'production': 'idf'}
['#', 0, 'begin', 1, 'VARDCLR', 3, 'let', 5, 'IDF', 20]
['#', 0, 'begin', 1, 'VARDCLR', 3, 'let', 5, 'IDF', 20, '=', 31]
['#', 0, 'begin', 1, 'VARDCLR', 3, 'let', 5, 'IDF', 20, '=', 31, 'num', 44]
['#', 0, 'begin', 1, 'VARDCLR', 3, 'let', 5, 'IDF', 20, '=', 31, 'num', 44, ';', 62]
{'non_term

In [ ]:
Do["number"]

In [44]:
get_reduction_rule(44)

"Pas d'action"



| Name | Description | age         
| :- |:------------- | :-
|Mary| She is a nice girl.  | 20
| Jackie Junior | He is a very naughty boy. | 5

#### TODO : 3andi pb f l'expression arithmetique !!!!

In [36]:
def reduction(number):
    regle = get_reduction_rule(number)
    right_part = regle["production"].split(sep=" ")
    right_part.reverse()
    left_part = regle["non_terminal"]
    # faire la reduction 
    while right_part != []:
        if Stack[-1] != right_part[0]: # Stack[-1] signifie sommet de pile 
            Stack.pop() # depiler
        elif Stack[-1] == right_part[0]:
            Stack.pop()
            right_part.pop(0)
        else :
            sys.exit(1)
    # get la relation entre Stack[-1] et le non-terminal     
    rel = lr_table[left_part][int(Stack[-1])]
    # empiler la partie gauche :
    Stack.append(left_part)
    # empiler la relation
    Stack.append(int(rel)) 

# Passage vers un language intermédiare

In [125]:
prog = "begin let idf = num ; let idf = num ; >> idf ; while ( idf <= idf ) : idf = idf + idf ; idf = idf ; idf = idf ; idf = idf + num ; ewhile << idf ; end"

In [126]:
source = prog.split(sep=" ")

In [127]:
source

['begin',
 'let',
 'idf',
 '=',
 'num',
 ';',
 'let',
 'idf',
 '=',
 'num',
 ';',
 '>>',
 'idf',
 ';',
 'while',
 '(',
 'idf',
 '<=',
 'idf',
 ')',
 ':',
 'idf',
 '=',
 'idf',
 '+',
 'idf',
 ';',
 'idf',
 '=',
 'idf',
 ';',
 'idf',
 '=',
 'idf',
 ';',
 'idf',
 '=',
 'idf',
 '+',
 'num',
 ';',
 'ewhile',
 '<<',
 'idf',
 ';',
 'end']

In [128]:
c_source = ""
while source != []:
    item = source.pop(0)
    
    if item == "begin":
        c_source += """#include<stdio.h> 
        int main(){"""
    if item == "end":
        c_source += "return 0;}"
    if item =="if":
        c_source += "if"
    if item =="eif":
        c_source += "}"
    if item =="else":
        c_source += "else}"
    if item =="eelse":
        c_source += "}"
    if item =="while":
        c_source += "while"       
    if item =="ewhile":
        c_source += "}"
    # declaration
    if item =="let":
        c_source += "int"
    if item =="idf":
        c_source +="v_idf" 
    if item =="num":
        c_source +="num" 
    # ecriture ;
    if item =="<<":
        c_source += 'printf("%d",' + source.pop(0) + ")"
    # lire ;
    if item ==">>":
        c_source += 'scanf("%d",&' + source.pop(0) + ")"
    # sep 
    if item ==";":
        c_source += ";"
    if item ==":":
        c_source += "{"
    if item =="(":
        c_source += "("
    if item ==")":
        c_source += ")"
    # opérations mathematiques
    if item =="=":
        c_source += "="
    if item =="+":
        c_source += "+"
    if item =="-":
        c_source += "-"
    if item =="/":
        c_source += "/"
    if item =="*":
        c_source += "*"
    if item =="%":
        c_source += "%"
        # comparaison :
    if item =="<":
        c_source += "<"
    if item ==">":
        c_source += ">"
    if item =="~":
        c_source += "=="
    if item =="<=":
        c_source += "<="
    if item ==">=":
        c_source += ">="
f = open("c.c","w")
f.write(c_source)
f.close()

In [129]:
c_source

'#include<stdio.h> \n        int main(){intv_idf=num;intv_idf=num;scanf("%d",&idf);while(v_idf<=v_idf){v_idf=v_idf+v_idf;v_idf=v_idf;v_idf=v_idf;v_idf=v_idf+num;}printf("%d",idf);return 0;}'

In [135]:
import os

compilez = 'gcc c_souce.c -o output'
os.system(compilez)


os.system("gnome-terminal -e 'bash -c \"ls; exec bash\"'")

0

In [137]:
used = [
    {'token_type': 'begin', 'value': None},
    {'token_type': 'let', 'value': None},
    {'token_type': 'idf', 'value': '_a'},
    {'token_type': '=', 'value': None},
    {'token_type': 'num', 'value': '1'},
    {'token_type': ';', 'value': None},
    {'token_type': 'end', 'value': None}]

In [144]:
a = [ item["value"] for item in used ]
b = [ item["token_type"] for item in used ]

In [145]:
a

[None, None, '_a', None, '1', None, None]

In [146]:
b

['begin', 'let', 'idf', '=', 'num', ';', 'end']

In [ ]:
c_source = ""
for i in range(len(source)) :
    if source[i] == "begin":
        c_source += """#include<stdio.h> 
        int main(){"""
    if source[i] == "end":
        c_source += "return 0;}"
    if source[i] =="if":
        c_source += "if"
    if source[i] =="eif":
        c_source += "}"
    if source[i] =="else":
        c_source += "else}"
    if source[i] =="eelse":
        c_source += "}"
    if source[i] =="while":
        c_source += "while"       
    if source[i] =="ewhile":
        c_source += "}"
    # declaration
    if source[i] =="let":
        c_source +="'type '" # regard le type du svt
    
    if source[i] =="idf":
        c_source +="v_idf" 
    if source[i] =="num":
        c_source +="num" 
    # ecriture ;
    if source[i] =="<<":
        c_source += "printf('type'," + source[i+1] + ")"
    # lire ;
    if source[i] ==">>":
        c_source += "scanf('type'," + source[i+1] + ")"
    # sep 
    if source[i] ==";":
        c_source += ";"
    if source[i] ==":":
        c_source += "{"
    if source[i] =="(":
        c_source += "("
    if source[i] ==")":
        c_source += ")"
    # opérations mathematiques
    if source[i] =="=":
        c_source += "="
    if source[i] =="+":
        c_source += "+"
    if source[i] =="-":
        c_source += "-"
    if source[i] =="/":
        c_source += "/"
    if source[i] =="*":
        c_source += "*"
    if source[i] =="%":
        c_source += "%"
        # comparaison :
    if source[i] =="<":
        c_source += "<"
    if source[i] ==">":
        c_source += ">"
    if source[i] =="~":
        c_source += "=="
    if source[i] =="<=":
        c_source += "<="
    if source[i] ==">=":
        c_source += ">="
#f = open("c.c","w")
#f.write(c_source)
#f.close()